# BERT Meets Cranfield - Enrichment and Transfer Learning Approach
The BM25 step finds a significant portion of the relevant documents for each query, but not all. The following notebooks implements a function that enriches the training set to find wether it would be benficial include those missed documents.

In [5]:
%cd /content/drive/MyDrive/COMPUTING SCIENCE/THESIS_PROJECT/bert-meets-cranfield-enrich/Code
# %cd /home/jupyter/BERT-BM25-Thesis-Project/bert-meets-cranfield-enrich/Code

/content/drive/MyDrive/COMPUTING SCIENCE/THESIS_PROJECT/bert-meets-cranfield-enrich/Code


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip3 install -r ../requirements.txt

     |████████████████████████████████| 28.5 MB 43 kB/s 
     |████████████████████████████████| 2.8 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 636 kB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


## Import

In [8]:
import utils
import data_utils
from operator import itemgetter
import os
import numpy as np

import torch
import importlib
# from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM, BertForNextSentencePrediction

import timeit

### Import Refresh
When a supporting py-file (such as utils.py) is changed, this code will have the lib reloaded while not reloading the entire notebook.

In [9]:
# call after making any changes in utils.py
importlib.reload(utils) 
importlib.reload(data_utils)

<module 'data_utils' from '/content/drive/My Drive/COMPUTING SCIENCE/THESIS_PROJECT/bert-meets-cranfield-enrich/Code/data_utils.py'>

## Set hyper-paramters and test settings

In [13]:

# ========================================
#               Hyper-Parameters
# ========================================
SEED = 76
MODE = 'Re-ranker'
MODEL_TYPE = 'bert-base-uncased'
LEARNING_RATE = 2e-5
MAX_LENGTH = 128
BATCH_SIZE = 32
EPOCHS = 1
TOP_BM25 = 100
MAP_CUT = 100
NDCG_CUT = 20
if MODE == 'Full-ranker':
    TEST_BATCH_SIZE = 1400
else:
    TEST_BATCH_SIZE = 100

# Set the seed value all over the place to make this reproducible.
utils.initialize_random_generators(SEED)

BM25_ENRICH = 'add' # or 'add' or 'swap' (default=no enrichment of BM25 results)

### Enriching function for BM25 results

In [14]:
def get_bm25_plus_other_rel(bm25_tn, labels, queries):
      bm25_top_n_rel_padded = [0]*len(queries) # a bm25_top_n list padded with the remaining relevant documents
      bm25_top_n_swap = [0]*len(queries) 
    
      for qi in range(len(queries)):
        # get the list of relelvant documents
        lbi = np.where(labels[qi] == 1)
        # note this numbering is only compatible with the labels list


        # get the list of bm25_top_n
        np_bm25_qi_docs = np.array(bm25_top_n[qi]) 

        # evaluate what relevant documents should be added
        pad_rel = np.setdiff1d(lbi, np_bm25_qi_docs)
        # if len(pad_rel) > 0:
        pad_rel = tuple(pad_rel)
        bm25_top_n_rel_padded[qi] = bm25_top_n[qi] + pad_rel
        # create a list with least relevant items swapped for unfound relevant
        for i in range(len(pad_rel)):
          # CHECK
          # are we to swap a relevant document?
          current_doc = np_bm25_qi_docs[-(i+1)] 
          
          if np.count_nonzero(current_doc == lbi) > 0:
            print('Relevant doc overwritten!')
          # CONTINUE  
          np_bm25_qi_docs[-(i+1)] = pad_rel[i]
          
        bm25_top_n_swap[qi] = np_bm25_qi_docs
      return bm25_top_n_rel_padded, bm25_top_n_swap

## Train and Test

In [16]:
if __name__ == "__main__":
    print("# ========================================")
    print("#               Hyper-Parameters")
    print(MODE)
    print(MODEL_TYPE)
    print(LEARNING_RATE)
    print(MAX_LENGTH)
    print(BATCH_SIZE)
    print(EPOCHS)
    print("# ========================================")
    print("#               Experiment-Settings")
    print('BM25_ENRICHMENT: ', BM25_ENRICH)

    print("# ========================================")
    print("#               Other")
    print(torch.cuda.get_device_name())
    print("# ========================================")
    
    start = timeit.default_timer()
    
    device = utils.get_gpu_device()
    if not os.path.exists('../Output_Folder'):
        os.makedirs('../Output_Folder')

    queries = data_utils.get_queries('../Data/cran/cran.qry')
    corpus = data_utils.get_corpus('../Data/cran/cran.all.1400')
    rel_fed = data_utils.get_judgments('../Data/cran/cranqrel')

    labels = utils.get_binary_labels(rel_fed)
    tokenized_corpus = [doc.split(" ") for doc in corpus]
    tokenized_queries = [query.split(" ") for query in queries]

    bm25, bm25_top_n = utils.get_bm25_top_results(tokenized_corpus, tokenized_queries, TOP_BM25)

    # no matter what BM25_ENRICH is, this line is needed to get `temp_feedback` for the test set
    padded_all, attention_mask_all, token_type_ids_all, temp_feedback = utils.bert_tokenizer(MODE, bm25_top_n, corpus,
                                                                                             labels, queries,
                                                                                             MAX_LENGTH, MODEL_TYPE)
    if BM25_ENRICH == 'swap':
        bm25_top_n_ext, bm25_top_n_swap = get_bm25_plus_other_rel(bm25_top_n, labels, queries)
        padded_all_swap, attention_mask_all_swap, token_type_ids_all_swap, temp_feedback_swap = utils.bert_tokenizer(MODE, bm25_top_n_swap, corpus,
                                                                                                                     labels, queries,
                                                                                                                     MAX_LENGTH, MODEL_TYPE)
    elif BM25_ENRICH == 'add':
        bm25_top_n_add, bm25_top_n_swap = get_bm25_plus_other_rel(bm25_top_n, labels, queries)
        padded_all_add, attention_mask_all_add, token_type_ids_all_add, temp_feedback_add = utils.bert_tokenizer(MODE, bm25_top_n_add, corpus,
                                                                                                                 labels, queries,
                                                                                                                 MAX_LENGTH, MODEL_TYPE)

    # ========================================
    #               Folds
    # ========================================
    mrr_bm25_list, map_bm25_list, ndcg_bm25_list = [], [], []
    mrr_bert_list, map_bert_list, ndcg_bert_list = [], [], []
    mrr_bm25, map_bm25, ndcg_bm25 = 0, 0, 0
    mrr_bert, map_bert, ndcg_bert = 0, 0, 0

    for fold_number in range(1, 6):
        print('======== Fold {:} / {:} ========'.format(fold_number, 5))
        train_index, test_index = data_utils.load_fold(fold_number)

        padded, attention_mask, token_type_ids = [], [], []
        if MODE == 'Re-ranker':
            # no matter BM25_ENRICH-mode, next line required for test set construction
            padded, attention_mask, token_type_ids = padded_all, attention_mask_all, token_type_ids_all
            if BM25_ENRICH == 'swap':
                padded_swap, attention_mask_swap, token_type_ids_swap = padded_all_swap, attention_mask_all_swap, token_type_ids_all_swap
            elif BM25_ENRICH == 'add':
                padded_add, attention_mask_add, token_type_ids_add = padded_all_add, attention_mask_all_add, token_type_ids_all_add
            
        else:
            temp_feedback = []
            for query_num in range(0, len(bm25_top_n)):
                if query_num in test_index:
                    doc_nums = range(0, 1400)
                else:
                    doc_nums = bm25_top_n[query_num]
                padded.append(list(itemgetter(*doc_nums)(padded_all[query_num])))
                attention_mask.append(list(itemgetter(*doc_nums)(attention_mask_all[query_num])))
                token_type_ids.append(list(itemgetter(*doc_nums)(token_type_ids_all[query_num])))
                temp_feedback.append(list(itemgetter(*doc_nums)(labels[query_num])))

        # Enricht the training set (or keep default)
        if BM25_ENRICH == 'default':
            train_dataset = data_utils.get_tensor_dataset(train_index, padded, attention_mask, token_type_ids,
                                                          temp_feedback)
        elif BM25_ENRICH == 'swap':
            train_dataset = data_utils.get_tensor_dataset(train_index, padded_swap, attention_mask_swap, token_type_ids_swap,
                                                    temp_feedback_swap)
        elif BM25_ENRICH == 'add':
            train_dataset = data_utils.get_tensor_dataset(train_index, padded_add, attention_mask_add, token_type_ids_add,
                                                    temp_feedback_add)

        test_dataset = data_utils.get_tensor_dataset(test_index, padded, attention_mask, token_type_ids, temp_feedback)

        mrr_bm25, map_bm25, ndcg_bm25, mrr_bm25_list, map_bm25_list, ndcg_bm25_list = utils.get_bm25_results(
            mrr_bm25_list, map_bm25_list, ndcg_bm25_list, test_index, tokenized_queries, bm25, mrr_bm25, map_bm25,
            ndcg_bm25, rel_fed, fold_number, MAP_CUT, NDCG_CUT)

        train_dataloader, test_dataloader, model, optimizer, scheduler = utils.model_preparation(MODEL_TYPE, train_dataset,
                                                                                                 test_dataset,
                                                                                                 BATCH_SIZE, TEST_BATCH_SIZE,
                                                                                                 LEARNING_RATE, EPOCHS)
        # ========================================
        #               Training Loop
        # ========================================
        epochs_train_loss, epochs_val_loss = [], []
        for epoch_i in range(0, EPOCHS):
            # ========================================
            #               Training
            # ========================================
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
            print('Training...')
            model, optimizer, scheduler = utils.training(model, train_dataloader, device, optimizer, scheduler)
        # ========================================
        #               Testing
        # ========================================
        print('Testing...')
        mrr_bert, map_bert, ndcg_bert, mrr_bert_list, map_bert_list, ndcg_bert_list = utils.testing(MODE, model,
                                                                                                    test_dataloader,
                                                                                                    device, test_index,
                                                                                                    bm25_top_n,
                                                                                                    mrr_bert_list,
                                                                                                    map_bert_list,
                                                                                                    ndcg_bert_list,
                                                                                                    mrr_bert, map_bert,
                                                                                                    ndcg_bert, rel_fed,
                                                                                                    fold_number,
                                                                                                    MAP_CUT, NDCG_CUT)
    print("  BM25 MRR:  " + "{:.4f}".format(mrr_bm25 / 5))
    print("  BM25 MAP:  " + "{:.4f}".format(map_bm25 / 5))
    print("  BM25 NDCG: " + "{:.4f}".format(ndcg_bm25 / 5))

    print("  BERT MRR:  " + "{:.4f}".format(mrr_bert / 5))
    print("  BERT MAP:  " + "{:.4f}".format(map_bert / 5))
    print("  BERT NDCG: " + "{:.4f}".format(ndcg_bert / 5))

    utils.t_test(mrr_bm25_list, mrr_bert_list, 'MRR')
    utils.t_test(map_bm25_list, map_bert_list, 'MAP')
    utils.t_test(ndcg_bm25_list, ndcg_bert_list, 'NDCG')
    
    stop = timeit.default_timer()
    wall_time = (stop - start) / 60 

    print('Time: ', wall_time, ' min') 

    # utils.results_to_csv('./mrr_bm25_list.csv', mrr_bm25_list)
    # utils.results_to_csv('./mrr_bert_list.csv', mrr_bert_list)
    # utils.results_to_csv('./map_bm25_list.csv', map_bm25_list)
    # utils.results_to_csv('./map_bert_list.csv', map_bert_list)
    # utils.results_to_csv('./ndcg_bm25_list.csv', ndcg_bm25_list)
    # utils.results_to_csv('./ndcg_bert_list.csv', ndcg_bert_list)

# ========================================
#               Hyper-Parameters
Re-ranker
bert-base-uncased
2e-05
128
32
1
# ========================================
#               Experiment-Settings
BM25_ENRICHMENT:  add
# ========================================
#               Other
Tesla K80
# ========================================
GPU Type: Tesla K80


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
Relevant doc overwritten!
======== Fold 1 / 5 ========
MRR:  0.7837
MAP:  0.3493
NDCG: 0.5011
45


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

======== Epoch 1 / 1 ========
Training...


AttributeError: ignored